In [72]:
from py2neo import Graph, Node, Relationship
graph = Graph("http://localhost:7474/db/data", user="bansri", password="slack")
graph.delete_all()

In [73]:
import random
from faker import Faker
import faker.providers
from faker.providers import BaseProvider
from faker import Factory
fake = Factory.create()
from random import expovariate, normalvariate, choice, randint, shuffle

In [74]:
import random
domains = [ "hotmail.com", "gmail.com", "aol.com", "mail.com" , "mail.kz", "yahoo.com"]
letters = ["a", "b", "c", "d","e", "f", "g", "h", "i", "j", "k", "l"]

def get_one_random_domain(domains):
        return domains[random.randint( 0, len(domains)-1)]


def get_one_random_name(letters):
    email_name = ""
    for i in range(7):
        email_name = email_name + letters[random.randint(0,11)]
    return email_name

def generate_random_emails():
    one_name = str(get_one_random_name(letters))
    one_domain = str(get_one_random_domain(domains))         
    return one_name  + "@" + one_domain
        
        
def gen_gender():
    '''Return random gender M/F
    '''
    return choice('FM')

status_list = ['active', 'In a meeting', 'commuting', 'sick', 'On holiday', 'Working remotely']
def pick_status():
    return status_list[random.randint(0,len(status_list)-1)]

In [75]:
generate_random_emails()

'bfbfdjf@yahoo.com'

In [76]:
#users have name, phone, email
users = []
def generate_users():
    ''' Generate fields for one employee
    '''
    
        
    
    
    new_user = Node("USER", fname=fake.first_name() , lname=fake.last_name(), email_id= generate_random_emails(), phone_num = fake.phone_number(),status = pick_status())
    users.append(new_user)
    graph.create(new_user)
    
    return new_user

In [77]:
# Workspace have name, date_of_creation
workspace_num = 10
used_workspaces = {}

def generate_workspaces():
    global workspace_num
    unique = False
    while not unique:
        wname = fake.company()
        try:
            used_workspaces[wname]
        except KeyError:
            unique = True
    used_workspaces[wname] = True
    workspace_num = workspace_num + 1
    ws = Node("WORKSPACE", wname=wname, date_of_creation=fake.date())
    workspaces.append(ws)
    graph.create(ws)
    return ws

In [78]:
#Channels have name and start_date
channel_num = 100
channels = []
def generate_channels():
    global channel_num
    cname = "CHANNEL" + str(channel_num)
    channel_num = channel_num + 1
    channel = Node("CHANNEL", cname = fake.company_suffix(), start_date=fake.date()) 
    channels.append(channel)
    graph.create(channel)
    return channel

In [79]:
#Posts(messages) have post, post_date
post_num=200
posts = []
def generate_posts():
    global post_num
    pname="POST" + str(post_num)
    post_num = post_num + 1
    post = Node("POST", pname = fake.word(), post_date=fake.text(max_nb_chars=50, ext_word_list=None))
    posts.append(post)
    graph.create(post)
    return post

In [80]:
#generating the data

for i in range(100):
    generate_users()

for i in range(10):
    generate_workspaces()

for i in range(20):
    generate_channels()
    
for i in range(1000):
    generate_posts()

print (len(users), len(workspaces), len(channels), len(posts))

100 18 20 1000


In [81]:
#creating relations

relations = []

for user in users:
    workspace = random.choice(workspaces)
    rel = Relationship(user, "HAS_ACCESS_TO", workspace)
    relations.append(rel)
    

for channel in channels:
    worksapce = random.choice(workspaces)
    rel = Relationship(worksapce, "HAS_A", channel)
    relations.append(rel)
    
for post in posts:
    channel = random.choice(channels)
    user = random.choice(users)
    rel = Relationship(post, "IS_IN", channel)
    rel1 = Relationship(user,"POSTED",post)
    relations.append(rel)
    relations.append(rel1)




In [82]:
for relation in relations:
    graph.create(relation)